In [3]:
import argparse
import torch
import numpy as np
import os
import time
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from dgtv.dgtv import *
import pickle
import logging
import sys
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler



In [2]:
# !python train_gtv.py --batch 1 --lr 1e-3 --umin 0.01 --umax 100 -m model/GTV.pkl --epoch 5 --train dataset/dataset_structure --width 36

In [ ]:
def supporting_matrix(opt):
    dtype = opt.dtype
    cuda = opt.cuda
    width = opt.width

    pixel_indices = [i for i in range(width * width)]
    pixel_indices = np.reshape(pixel_indices, (width, width))
    A = connected_adjacency(pixel_indices, connect=opt.connectivity)
    A_pair = np.asarray(np.where(A.toarray() == 1)).T
    A_pair = np.unique(np.sort(A_pair, axis=1), axis=0)

    opt.edges = A_pair.shape[0]
    H_dim0 = opt.edges
    H_dim1 = width ** 2

    I = torch.eye(width ** 2, width ** 2).type(dtype)
    A = torch.zeros(width ** 2, width ** 2).type(dtype)
    H = torch.zeros(H_dim0, H_dim1).type(dtype)
    for e, p in enumerate(A_pair):
        H[e, p[0]] = 1
        H[e, p[1]] = -1
        A[p[0], p[1]] = 1

    opt.I = I
    opt.pairs = A_pair
    opt.H = H
    opt.connectivity_full = A.requires_grad_(True)
    opt.connectivity_idx = torch.where(A > 0)

    for e, p in enumerate(A_pair):
        A_temp = A.clone()
        A_temp[p[1], p[0]] = 1
        A = A_temp
    opt.logger.info("OPT created on cuda: {0} {1}".format(cuda, dtype))


In [5]:
def test(
    seed, model_name, cont=None, optim_name=None, subset=None, epoch=100, args=None
):
    cuda = True if torch.cuda.is_available() else False
    torch.autograd.set_detect_anomaly(True)
    opt.logger.info("CUDA: {0}".format(cuda))
    if cuda:
        dtype = torch.cuda.FloatTensor
        opt.logger.info(torch.cuda.get_device_name(0))
    else:
        dtype = torch.FloatTensor

    DST = "./"
    DST = ""
    PATH = os.path.join(DST, model_name)
    SAVEPATH = PATH.split(".")[-1]
    SAVEDIR = "".join(PATH.split(".")[:-1]) + "_"
    batch_size = opt.batch_size
    if not subset:
        _subset = ["10", "1", "7", "8", "9"]
        # _subset = ["1", "3", "5", "7", "9"]
        opt.logger.info("Train: {0}".format(_subset))
        subset = [i + "_" for i in _subset]
    else:
        subset = [i + "_" for i in subset]
    dataset = RENOIR_Dataset(
        img_dir=os.path.join(opt.train),
        transform=transforms.Compose(
            [standardize(normalize=False), ToTensor()]),
        subset=None,
    )
    opt.logger.info("Splitting patches...")
    patch_splitting(
        dataset=dataset, output_dst="tmp", patch_size=args.width, stride=args.width / 2
    )
    dataset = RENOIR_Dataset(
        img_dir=os.path.join("tmp", "patches"),
        transform=transforms.Compose(
            [standardize(normalize=False), ToTensor()]),
        subset=subset,
    )

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=4,
        drop_last=True,
    )

    supporting_matrix(opt)

In [6]:
def supporting_matrix(opt):
    dtype = opt.dtype
    cuda = opt.cuda
    width = opt.width

    pixel_indices = [i for i in range(width * width)]
    pixel_indices = np.reshape(pixel_indices, (width, width))
    A = connected_adjacency(pixel_indices, connect=opt.connectivity)
    A_pair = np.asarray(np.where(A.toarray() == 1)).T
    A_pair = np.unique(np.sort(A_pair, axis=1), axis=0)

    opt.edges = A_pair.shape[0]
    H_dim0 = opt.edges
    H_dim1 = width ** 2

    I = torch.eye(width ** 2, width ** 2).type(dtype)
    A = torch.zeros(width ** 2, width ** 2).type(dtype)
    H = torch.zeros(H_dim0, H_dim1).type(dtype)
    for e, p in enumerate(A_pair):
        H[e, p[0]] = 1
        H[e, p[1]] = -1
        A[p[0], p[1]] = 1

    opt.I = I
    opt.pairs = A_pair
    opt.H = H
    opt.connectivity_full = A.requires_grad_(True)
    opt.connectivity_idx = torch.where(A > 0)

    for e, p in enumerate(A_pair):
        A_temp = A.clone()
        A_temp[p[1], p[0]] = 1
        A = A_temp
    opt.logger.info("OPT created on cuda: {0} {1}".format(cuda, dtype))


In [ ]:


# Đoạn code để tạo một tập dữ liệu giả định từ GTV
# Đảm bảo rằng data_features và data_labels được xây dựng từ đặc trưng GTV và nhãn tương ứng.

# Xác định đặc trưng và nhãn từ dữ liệu GTV sử dụng supporting_matrix(opt)
opt = ...  # Đối tượng opt được sử dụng để gọi supporting_matrix(opt)
data_features = supporting_matrix(opt)  # Dùng hàm supporting_matrix để trích xuất đặc trưng
data_labels = [...]  # Nhãn tương ứng

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(data_features, data_labels, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Xây dựng mô hình KNN với K=3 (có thể điều chỉnh K theo nhu cầu)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Dự đoán nhãn của dữ liệu kiểm tra
y_pred = knn.predict(X_test)

# Đánh giá hiệu suất của mô hình KNN
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
